## Import

In [5]:
# Import of basic packages
import numpy as np
import pandas as pd
import operator
import warnings

warnings.filterwarnings('ignore')

# Import of chart packages
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt

# Import of machine learning packages
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingRegressor, StackingClassifier, AdaBoostClassifier
#from xgboost import XGBClassifier
from sklearn.svm import SVC

# Set random seed 
RSEED = 0

# Import of machine learning metric packages
from sklearn.metrics import f1_score, classification_report, confusion_matrix, mean_squared_error, r2_score, accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve, fbeta_score
from sklearn import metrics


## Load

## Explore data analysis

## Data cleaning

## Feature engineering

## Splitting data for testing 

## Trainining the model

## Evaluation

## Validation

## Packaging